In [1]:
import nltk

from nltk.corpus import stopwords
from collections import Counter

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn import datasets, model_selection, linear_model, metrics

import pickle

import os

# 추가한 라이브러리
import re
import joblib

In [2]:
#open 피클 
with open("gmae_pkl/game_df_b50.pkl","rb") as fr:
    game_df_b50 = pickle.load(fr)

In [3]:
#불용어 사전
stop_words = stopwords.words("english")

In [4]:
#불용어 추가
stop_words.append(',')
stop_words.append('(')
stop_words.append(')')
stop_words.append('.')
stop_words.append('!')
stop_words.append("'")
stop_words.append("`")
stop_words.append('"')

# 불용어 삭제
#  stop_words.append('\\')

In [5]:
# stop_words

In [6]:
#모든 리뷰 꺼내며 불용어 처리
b_games_review = []
for k in range(len(game_df_b50)):   
    game_review = ''
    for i in range(100):
    
        try:
            game_review = game_review + game_df_b50.iloc[k][i]['review']

        except:
            pass
    
    sentence = game_review
    
    #문장 토큰화
    tokens = nltk.word_tokenize(sentence)
    
    # 불용어 처리
    result = []
    for token in tokens:
        if token.lower() not in stop_words:
            result.append(token.lower())
    
    # 문장 재봉합
    game_review = ''
    for k in result:
        game_review = game_review + ' ' + k

            
    b_games_review.append(game_review)

In [7]:
b_games_review[0]

" easily one best well known ds releases year includes beautiful plot hints throughout challenging enough keep interested without becoming game random clicking ost incredible shu takumi done excellent job game aesthetics concept 'm hoping see sequel form1hands one best games ds ghost trick completely brilliant start finish- great puzzles fantastic characters engaging story truly awesome ending character fully realised everyone part play stellar tale puzzles bit hit miss occassionally 'll forgive 's utterly likeable.innovative addictive style gameplay engaging story rich creativity without cliche catchy music fits situation game cast unforgettable characters eccentric personalities ghost trick truly heartfelt magical experience proves amazing handheld video game make someone feel shows nintendo ds still worthy console.phenomenal game perfect blend puzzle solving mystery creative gameplay characters charming story captivating puzzles generally well done strange ones love good mystery gam

In [8]:
len(b_games_review)

50

In [9]:
# ※정규식 필터링 추가

review_word_pattern = re.compile(r'[A-Za-z]+')

# 복잡해 보이지만 로직은 별거 없고
# b_~ 리스트를 순회하며 각 문자열을 정규식 적용하여 단어 분리
# 그 후 길이가 3 이상인 단어만 뽑아 묶은 문자열로 교체
b_games_review = [' '.join([word if len(word) > 2 else '' \
                            for word in re.findall(review_word_pattern, review)]) \
                  for review in b_games_review]

In [10]:
b_games_review[0]

'easily one best well known  releases year includes beautiful plot hints throughout challenging enough keep interested without becoming game random clicking ost incredible shu takumi done excellent job game aesthetics concept  hoping see sequel form hands one best games  ghost trick completely brilliant start finish great puzzles fantastic characters engaging story truly awesome ending character fully realised everyone part play stellar tale puzzles bit hit miss occassionally  forgive  utterly likeable innovative addictive style gameplay engaging story rich creativity without cliche catchy music fits situation game cast unforgettable characters eccentric personalities ghost trick truly heartfelt magical experience proves amazing handheld video game make someone feel shows nintendo  still worthy console phenomenal game perfect blend puzzle solving mystery creative gameplay characters charming story captivating puzzles generally well done strange ones love good mystery game something mus

In [11]:
vectorizer = TfidfVectorizer()

In [12]:
X = vectorizer.fit_transform(b_games_review)

In [13]:
pd.DataFrame(cosine_similarity(X)) # 정규식 적용하지 않은 노트북 파일의 출력결과와 비교

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.000000,0.255440,0.197933,0.334817,0.470900,0.396878,0.465984,0.217807,0.387585,0.247298,...,0.312475,0.493912,0.269332,0.090812,0.487698,0.450977,0.211918,0.408320,0.294014,0.419507
1,0.255440,1.000000,0.129237,0.170482,0.288347,0.228123,0.289787,0.093690,0.209834,0.136913,...,0.156342,0.252566,0.154468,0.054964,0.289391,0.265425,0.175502,0.241434,0.167221,0.264807
2,0.197933,0.129237,1.000000,0.145837,0.234106,0.221682,0.267592,0.115994,0.215292,0.306694,...,0.133568,0.208097,0.136100,0.048535,0.238450,0.178089,0.104964,0.223614,0.150380,0.197087
3,0.334817,0.170482,0.145837,1.000000,0.351574,0.299893,0.346949,0.181862,0.298381,0.183110,...,0.847233,0.353670,0.198870,0.072966,0.405015,0.363312,0.152590,0.314363,0.220089,0.315376
4,0.470900,0.288347,0.234106,0.351574,1.000000,0.479356,0.604059,0.219552,0.453595,0.263945,...,0.318466,0.532327,0.477449,0.293902,0.582580,0.504734,0.257199,0.499406,0.348546,0.846124
5,0.396878,0.228123,0.221682,0.299893,0.479356,1.000000,0.762917,0.390203,0.736604,0.406566,...,0.280329,0.428980,0.297801,0.111419,0.480228,0.394459,0.193391,0.781246,0.603558,0.440299
6,0.465984,0.289787,0.267592,0.346949,0.604059,0.762917,1.000000,0.385076,0.762237,0.388827,...,0.312302,0.522242,0.351013,0.138248,0.577201,0.471740,0.238456,0.801001,0.558875,0.558815
7,0.217807,0.093690,0.115994,0.181862,0.219552,0.390203,0.385076,1.000000,0.432984,0.165087,...,0.177155,0.220678,0.132793,0.041292,0.262746,0.193355,0.095116,0.380059,0.296324,0.198742
8,0.387585,0.209834,0.215292,0.298381,0.453595,0.736604,0.762237,0.432984,1.000000,0.323116,...,0.279844,0.409713,0.279797,0.102134,0.462916,0.383508,0.180650,0.744182,0.599392,0.417569
9,0.247298,0.136913,0.306694,0.183110,0.263945,0.406566,0.388827,0.165087,0.323116,1.000000,...,0.174207,0.248189,0.173797,0.057741,0.268939,0.216434,0.138623,0.345513,0.281365,0.241194


In [14]:
game_df_b50.iloc[44]

platform                                              Xbox One
title                  NieR: Automata - Become as Gods Edition
publisher                                          Square Enix
0            {'score': 9, 'review': 'I won’t mind admitting...
1            {'score': 10, 'review': 'Square enix did it ye...
                                   ...                        
95           {'score': 10, 'review': 'I love this game so m...
96           {'score': 9, 'review': 'An excellent game with...
97           {'score': 9, 'review': 'THE STORY AND THE MUSI...
98           {'score': 10, 'review': 'This is the most uniq...
99           {'score': 10, 'review': 'Beautiful soundtrack,...
Name: 44, Length: 103, dtype: object

In [15]:
game_df_b50.iloc[0]

platform                                                    DS
title                           Ghost Trick: Phantom Detective
publisher                                               Capcom
0            {'score': 10, 'review': 'Easily one of the bes...
1            {'score': 9, 'review': 'Hands down, one of the...
                                   ...                        
95                                                         NaN
96                                                         NaN
97                                                         NaN
98                                                         NaN
99                                                         NaN
Name: 0, Length: 103, dtype: object

In [16]:
# 유사도가 일정치를 넘는 결과 보기

cos_array = cosine_similarity(X)
threshold = 0.6

for i in range(50):
    for j in range(i+1, 50):
        if cos_array[i][j] > threshold:
            print(game_df_b50.at[i, 'title'], game_df_b50.at[i, 'platform'], '||',game_df_b50.at[j, 'title'], game_df_b50.at[j, 'platform'])

Final Fantasy XIV: Endwalker PC || Final Fantasy XIV: Shadowbringers PC
Final Fantasy XIV: Endwalker PC || Final Fantasy XIV: Endwalker PlayStation 5
Vanquish PlayStation 3 || Resident Evil 4 PlayStation 2
Vanquish PlayStation 3 || Vanquish PC
Vanquish PlayStation 3 || Resident Evil 4 GameCube
Vanquish PlayStation 3 || Vanquish Xbox 360
Resident Evil PlayStation || Resident Evil 4 PlayStation 2
Resident Evil PlayStation || Resident Evil 2 PlayStation
Resident Evil PlayStation || The Evil Within 2 Xbox One
Resident Evil PlayStation || Resident Evil 4 GameCube
Resident Evil PlayStation || Resident Evil (2002) GameCube
Resident Evil PlayStation || Resident Evil 3: Nemesis PlayStation
Resident Evil 4 PlayStation 2 || Resident Evil 2 PlayStation
Resident Evil 4 PlayStation 2 || It Takes Two PlayStation 5
Resident Evil 4 PlayStation 2 || The Evil Within 2 Xbox One
Resident Evil 4 PlayStation 2 || The Last of Us Remastered PlayStation 4
Resident Evil 4 PlayStation 2 || The Last of Us PlayStat